<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons Licence" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">COMP5611M - Building a Machine Learning Pipeline (part 2)</span> by <span xmlns:cc="http://creativecommons.org/ns#" property="cc:attributionName">Marc de Kamps and University of Leeds</span> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License</a>.

## Building a machine learning (part 2)

### Objectives

We will clean the data, in particular, we will
- Impute missing values
- Replace alpha-numeric information by one-hot-encoded vectors to arrive at a purely numerical implementation
- Add features to the dataset that are likely to boost prediction power
- We will use Transformer and Pipeline objects to provide self-documenting implementations of these steps

### Caveat

There is one small error in this notebook that we will leave in place, but have removed in (Part 3). Once you have run (Part 3), see if you can discover it.
**Warning** Use Part 3 as a basis for further coding, not this notebook (part 2).

We start where we left off in the previous notebook:

In [161]:
import os
import numpy as np
import pandas as pd
import tarfile

from sklearn.model_selection import StratifiedShuffleSplit

local_path = 'datasets/housing'


def restore():
    housing_tgz=tarfile.open(os.path.join(local_path,'./housing.tgz'))
    housing_tgz.extractall(path=local_path)
    housing_tgz.close()

    csv_path=os.path.join(local_path,'./housing.csv')
    housing = pd.read_csv(csv_path)

    # create test training set with stratified sampling (see previous notebook)
    housing["income_category"]=np.ceil(housing["median_income"]/1.5)
    housing["income_category"].where(housing["income_category"] < 5, 5.0, inplace = True)

    split = StratifiedShuffleSplit(n_splits=1, test_size=0.2,random_state=42)

    for train_index, test_index in split.split(housing,housing["income_category"]):
        strat_train_set = housing.loc[train_index]
        strat_test_set = housing.loc[test_index]
    
    for set_ in (strat_train_set, strat_test_set):
        set_.drop(("income_category"),axis=1,inplace=True)
        

    housing.drop(("income_category"),axis=1,inplace=True)

   
    return housing, strat_train_set, strat_test_set

housing, strat_train_set, start_test_set = restore()

[[-122.23 37.88 41.0 ... 8.3252 452600.0 'NEAR BAY']
 [-122.22 37.86 21.0 ... 8.3014 358500.0 'NEAR BAY']
 [-122.24 37.85 52.0 ... 7.2574 352100.0 'NEAR BAY']
 ...
 [-121.22 39.43 17.0 ... 1.7 92300.0 'INLAND']
 [-121.32 39.43 18.0 ... 1.8672 84700.0 'INLAND']
 [-121.24 39.37 16.0 ... 2.3886 89400.0 'INLAND']]


## Missing data

In many real world data sets there is missing data. Doctors forget to register the temperature at some days, sensors are sometimes faulty and produce no sensible output, etc. Pandas is a considerable step up from comma separated files in that they have an explicit representation for a missing value: *NaN*.

In general, machine learning algorithms can't work with NaN, however and this means either removing data where NaN occurs or replacing them by a sensible default value, a process called *imputation*.

Here you have three choices:
-Drop districts where NaN occurs
-Drop the entire attribute (i.e. remove the entire column where NaN occurs)
-Impute

Many imputation strategies are simple: they replace the missing value by the median (or sometimes mean) of the attribute. This is potentially dangerous: if the pattern of missingness is not random, but has systematic causes it may be wrong to use this strategy. More sophisticated strategies, that we will not consider here, consist of using for example a random forest or decision tree to try and predict the missing values.

Pandas offers quick ways for implementing simple imputation:

In [162]:
#housing.dropna(subset=["total_bedrooms"]) # drop those districts for which the total_bedrooms value is missing
#housing.drop("total_bedrooms",axis=1) # drop the entire column

# impute missing values by the median
median=housing["total_bedrooms"].median()
housing["total_bedrooms"].fillna(median)

[[-122.23 37.88 41.0 ... 8.3252 452600.0 'NEAR BAY']
 [-122.22 37.86 21.0 ... 8.3014 358500.0 'NEAR BAY']
 [-122.24 37.85 52.0 ... 7.2574 352100.0 'NEAR BAY']
 ...
 [-121.22 39.43 17.0 ... 1.7 92300.0 'INLAND']
 [-121.32 39.43 18.0 ... 1.8672 84700.0 'INLAND']
 [-121.24 39.37 16.0 ... 2.3886 89400.0 'INLAND']]


*scikit-learn* itself also offers support for simple imputation strategies. And since this nicely dovetails with the use of pipelines, which is an important topic in this notebook, let's check it out.

In [115]:
from sklearn.impute import SimpleImputer

imputer=SimpleImputer(strategy="median")

It should be as simple as calling the fit method on the data frame. So:

            imputer.fit(housing)

You will find this will cause an exception to be thrown:

In [116]:
imputer.fit(housing)

ValueError: Cannot use median strategy with non-numeric data:
could not convert string to float: 'NEAR BAY'

**Exercise 1** Examine the data frame to see what causes this problem.

In [117]:
#! Sample answer 
housing.head()

# It is clear that ocean_proximity is a text attribute and that imputer tries to calculate a numerical value

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


**Exercise 2** The offending data itself does not need to be imputed. So it could be dropped, at least temporary, from the dataframe. Research how you can create a dataframe that doesn't contain the offending data, then fit the imputer to it. Once you have done this, print the

                imputer.statistics_
                
member variable.

In [164]:
#! Sample answer
housing_num =housing.drop("ocean_proximity",axis=1)
imputer.fit(housing_num)
print(imputer.statistics_)

[-1.1849e+02  3.4260e+01  2.9000e+01  2.1270e+03  4.3500e+02  1.1660e+03
  4.0900e+02  3.5348e+00  1.7970e+05]


In the previous question, you should have created a new dataframe that doesn't contain the 'ocean_proximity' attribute. Let's assume this reduced dataframe is called *housing_num* which now is guaranteed to contain numerical data only.

The fit method has not changed *housing_num* at all. It only has fed the data of the reduced dataframe to the imputer and allowed it to calculate median values for each of the columns. This is called *training* the imputer.
Now the imputer must be used to actually clean up the data of the *housing_num* frame. This is done by calling the imputer's *transform* method on *housing_num*. **Before proceeding, please ensure you have created the reduced dataframe and called it housing_num or the notebook will crash.**

In [71]:
X=imputer.transform(housing_num)

**Exercise 3** Examine the form of the data represented by X.

In [72]:
#! Sample answer
print(X)
print(X.shape)

# So just one big numpy array, but without Nan!

[[-1.2223e+02  3.7880e+01  4.1000e+01 ...  1.2600e+02  8.3252e+00
   4.5260e+05]
 [-1.2222e+02  3.7860e+01  2.1000e+01 ...  1.1380e+03  8.3014e+00
   3.5850e+05]
 [-1.2224e+02  3.7850e+01  5.2000e+01 ...  1.7700e+02  7.2574e+00
   3.5210e+05]
 ...
 [-1.2122e+02  3.9430e+01  1.7000e+01 ...  4.3300e+02  1.7000e+00
   9.2300e+04]
 [-1.2132e+02  3.9430e+01  1.8000e+01 ...  3.4900e+02  1.8672e+00
   8.4700e+04]
 [-1.2124e+02  3.9370e+01  1.6000e+01 ...  5.3000e+02  2.3886e+00
   8.9400e+04]]
(20640, 9)


**Exercise 4** Convert X back into a dataframe with the original column names (minus *ocean_proximity*)

In [73]:
#! Sample answer
housing_tr = pd.DataFrame(X,columns=housing_num.columns)
housing_tr.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


### Text and Categorical Variables

In general machine learning algorithms need numerical values to work and *scikit-learn* expects data in two dimensional numpy arrays (like X) in the previous example. It is therefore necessary to convert text which often indicates a categorical variable (like, '<1H OCEAN', 'INLAND', 'NEAR OCEAN' etc.) into numerical values. (An exception to this would be a project that tries to apply NLP to text fragments, but that is not the case here; the column
*ocean_proximity* clearly contains categories). It would be possible to convert  them into numerical values as in '<1H OCEAN' = 0, 'INLAND' = 1, etc.. There is a potential problem with this as there is an impcit concept of nearness in this coding: 0 and 1 are closer together than 1 and 4, for example. If used in a clustering algorithm this nearness may be used without this being intended by the investigator. A one-hot encoding does not have this problem.

A one-hot coding would code 5 categories as follows:
(1,0,0,0,0)
(0,1,0,0,0)
   ...
(0,0,0,0,1)

There is no unintended concept of proximity in this coding and it is to be preferred for coding categorical variables numerically.

Again *scikit-learn* offers a custom-made class for transforming textual categorical variables into one-hot encoded ones:

In [74]:
from sklearn.preprocessing import OneHotEncoder
enc=OneHotEncoder()
enc.fit(housing["ocean_proximity"].values.reshape(-1,1))
print(enc.categories_)

[array(['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'],
      dtype=object)]


*reshape*  is necessary because a transformer (such as Imputer or OneHotEncoder) expects a 2D numpy array. **housing["ocean_proximity"]** is a *pandas.Series*, hence the use of values. The -1 is a neat little syntactic trick to prevent having to know or calculate the length of the array.

In [75]:
enc.transform(housing["ocean_proximity"].values.reshape(-1,1)).toarray()

array([[0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.]])

## Transformers

*Imputer* and *OneHotEncoder* are so-called **transformers**. Whenever you start to realise that you need to do a lot of repetitive processing, you should either use a tranformer or design one of your own, in case *scikit-learn*  doesn't have one that is necessary for your processing.

You can design a class of your own that encapsulates your preprocessing code. All you need to do is provide functions: 

                fit()
                transform()
                fit_transform()
                
*fit_transform* simply first calls *fit* and then *transform*

As an example, lets add some preprocessing that adds the number of rooms per house hold and optionally also
the number of rooms per household. Earlier we have seen that the number rooms per house hold correlates quite strongly with the median house value and it may be a variable that we want to represent explicitly in the dataset, meaning that we have to transform it. Optionally, we may want the number of bedrooms per room to the dataset as well. We will make the transformer configurable so that later in your analysis pipeline you can investigate both options, simply by setting a binary flag, which then becomes a hyperparameter.

In [170]:
from sklearn.base import BaseEstimator

class CombinedAttributesAdder(BaseEstimator):

    def __init__(self, do_add_bedrooms_per_room = False):
        
        # simply a binary variable per room
        self.do_add_bedrooms_per_room = do_add_bedrooms_per_room
        
        # These are the column indices of the respective columns. OK for illustration purposes.
        # For more robust code you would want to extract these values from the DataFrame by name.
        self.rooms_ix      = 3
        self.bedrooms_ix   = 4
        self.population_ix = 5
        self.household_ix  = 6
        
    def fit(self, X, y=None):
        # We don't transform the target values here
        return self
    
    def transform(self, X, y=None):
        rooms_per_household = X[:,self.rooms_ix]/X[:,self.household_ix]
        population_per_household = X[:, self.population_ix]/ X[:,self.rooms_ix]
        if self.do_add_bedrooms_per_room:
            bedrooms_per_room = X[:,self.bedrooms_ix]/X[:,self.rooms_ix]
            return np.c_[X,rooms_per_household, population_per_household,bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]
        
attr_adder=CombinedAttributesAdder(do_add_bedrooms_per_room=True)
housing_extra_attribs=attr_adder.transform(housing.values)
print(housing_extra_attribs)


[[-122.23 37.88 41.0 ... 6.984126984126984 0.3659090909090909
  0.14659090909090908]
 [-122.22 37.86 21.0 ... 6.238137082601054 0.3382166502324271
  0.15579659106916466]
 [-122.24 37.85 52.0 ... 8.288135593220339 0.338104976141786
  0.12951601908657123]
 ...
 [-121.22 39.43 17.0 ... 5.20554272517321 0.44676131322094054
  0.21517302573203195]
 [-121.32 39.43 18.0 ... 5.329512893982808 0.39838709677419354
  0.21989247311827956]
 [-121.24 39.37 16.0 ... 5.254716981132075 0.4980251346499102
  0.22118491921005387]]


The extra attributes have now been added to the data, which are spit out as a 2D numpy array without further structure.  Ultimately, this is what you need if you want to do numerical processing and exactly what *scikit-learn* algorithms expect.

The *np.c_* is a numpy shorthand for grouping vectors into matrices. There are many ways of achieving the same effect, some people prefer *hstack* , *vstack* constructions.

In [77]:
a=np.array([1,2,3]).T
b=np.array([4,5,6]).T
print(np.c_[a,b])

print(np.vstack([a,b]).T)

[[1 4]
 [2 5]
 [3 6]]
[[1 4]
 [2 5]
 [3 6]]


### Feature Scaling

Most machine learning algorithms perform better if the numerical values of the attributes are of comparable magnitude. The total number of rooms varies between 0 and 39320, whereas income is between 0 and 15. Typically, the attributes - but not the target variable, here median house value - are rescaled. An exception are decision tree based methods where the data is taken as is.

#### Min-max scaling (normalisation)

The data are shifted and rescaled so that their minimum value corresponds to 0 and their maximum value corresponds to 1. *scikit-learn* provides the **MinMaxScaler** to achieve this. It can be recongured to other target ranges than $[0,1]$ where desired.

#### Standardisation
The mean is substracted of all data points and then divided by the variance so that a zero-mean unit variance distribution results.

There are pros and cons to each of these methods. Some machine learning algorithms (neural networks) expect input values to be within a certain range, meaning that you have to use min-max scaling. But a single outlier can compress the other data points to heap up around 0, something that doesn't happen to the same extent in standardisation.

*scikit-learn* provides the *StandardScalar* transformer to implement this method.



## Pipelines

Pipelines group transformers sequentially and provide a single access point for running the entire preprocessing chain. The transformations we have discussed so far crop up almost in every machine learning analysis. Each transformation step is usually simple, and it may be that initially you have cobbled an ad hoc preprocessing step to get going with analysis. Such code tends to obscure your programmes. If you find yourself bored because you're constantly writing relatively simple proeprocessing steps, you're not using pipelines properly. The individual
Transformers can be hidden in python modules that you can import. The Pipelines define a very short definition
of the preprocessing steps and are almost self-documenting. An example.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('imputer',SimpleImputer(strategy="median")),
    ('attribs_adder',CombinedAttributesAdder()),
    ('std_scaler',StandardScaler())
])

housing_num_tr = num_pipeline.fit_transform(housing_num)

This piece of code is very readable, moreover since imputing and scaling occur so often, similar pipelines can
often be constructed for other analyses, and the indvidual components can be re-used with a minimum of new programming effors. 

The names are practical for documentation, but also allow named access to the elements of the pipeline in case they're needed downstream in the analysis:

In [79]:
num_pipeline['imputer']

SimpleImputer(strategy='median')

When you call the pipeline *fit* method, it calls the *fit_transform* method of the elements of the pipeline in order. All but the last elements of the pipeline must be *transformers*, the last can be an *estimator* (an *estimator*) has *fit* method, but not necessarily a *transform* or *fit_transform* method. ).

### Combining Seperate Pipelines for Numerical and Categorical Values

We have seen that we need to handle numerical and categorical values differently. This effectively boils down to the creation of two pipelines that each run on mutually exclusive columns. We would like to run these pipelines in parallel and combine their joint output in one 2D numpy array which contains the conversion of all attributes to sensible numerical values.

To this end we will write a custom transformer that selects only certain columns of the dataset, and drops the other ones. We will use that transformer, called **DataFrameSelector** to create two parallel pipelines and then
combined the two resulting datasets.

In [123]:
class DataFrameSelector(BaseEstimator):
    
    def __init__(self, attribute_names):
        self.attribute_names= attribute_names
        
    def fit(self,X, y = None):
        return self
    
    def transform(self, X):
        return X[self.attribute_names].values


Note that although there appears to be no pandas code, this works because X is a panda DataFrame, which can take a list of column names and select the appropriate columns!

The two pipelines can now be created as follows **from the training set**.

In [124]:
num_attribs=list(housing) # using pandas to get the names of attributes in housing_num, which we made by dropping ocean_proximity
num_attribs.remove("ocean_proximity")
cat_attribs=["ocean_proximity"]

num_pipeline= Pipeline([
    ('selector', DataFrameSelector(num_attribs)),
    ('imputer',SimpleImputer(strategy="median")),
    ('attribs_adder',CombinedAttributesAdder()),
    ('std_scaler',StandardScaler())
])

cat_pipeline = Pipeline([
    ('selector',DataFrameSelector(cat_attribs)),
    ('one hot',OneHotEncoder())
])

These pipelines can be run independently ('in parallel'). Their results now need to be combined. *scikit-learn* offers the **FeatureUnion** for this purpose.

In [125]:
from sklearn.pipeline import FeatureUnion

full_pipeline = FeatureUnion(transformer_list=[
    ("num_pipeline",num_pipeline),
    ("cat_pipeline",cat_pipeline)
])

The whole pipeline can now be run in one go on the original *housing* DataFrame. 

In [127]:
housing.head()
#housing=strat_train_set.drop("median_house_value",axis=1)
#housing_labels=strat_train_set["median_house_value"]
housing_prepared = full_pipeline.fit_transform(housing)

print(housing_prepared.shape)
print(housing_prepared)

(16512, 15)
  (0, 0)	-1.1560428086829155
  (0, 1)	0.7719496164846016
  (0, 2)	0.7433308916510305
  (0, 3)	-0.49323393384425046
  (0, 4)	-0.4454382074687401
  (0, 5)	-0.6362114070375079
  (0, 6)	-0.4206984222235789
  (0, 7)	-0.6149374443958345
  (0, 8)	-0.31205451913809157
  (0, 9)	-0.05357520795873586
  (0, 10)	1.0
  (1, 0)	-1.1760248286103931
  (1, 1)	0.6596947951050618
  (1, 2)	-1.165317203353399
  (1, 3)	-0.9089665536785813
  (1, 4)	-1.036927797035715
  (1, 5)	-0.998331346653321
  (1, 6)	-1.0222270483369085
  (1, 7)	1.336459363533279
  (1, 8)	0.217683376839971
  (1, 9)	-0.054238379827919576
  (1, 10)	1.0
  (2, 0)	1.186849027813462
  (2, 1)	-1.3421828528300457
  (2, 2)	0.1866418639414053
  :	:
  (16509, 8)	0.3469341967290955
  (16509, 9)	-0.05997192431397317
  (16509, 11)	1.0
  (16510, 0)	0.7822131242821033
  (16510, 1)	-0.8510680092945644
  (16510, 2)	0.1866418639414053
  (16510, 3)	-0.30991876289367937
  (16510, 4)	-0.37484891488667316
  (16510, 5)	-0.05717803824588582
  (16510, 6)